In [2]:
!pip install llmsherpa langchain chromadb
!pip install sentence-transformers
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install langchainhub
!pip install huggingface_hub
!pip install gradio
!pip install langchain
!pip install!pip install transformers
!pip install chromadb
!pip install pdfplumber
!pip install -U langchain-community
!pip install "unstructured[all-docs]" langchain langchain_community \
 chromadb langchain-experimental
!pip install langchain_chroma


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 7.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127

In [1]:
import os

def lister_dossiers_fichiers(dossier):
    # Vérifier si le chemin spécifié est un répertoire
    if not os.path.isdir(dossier):
        print(f"Le chemin spécifié ({dossier}) n'est pas un répertoire valide.")
        return
    
    # Afficher les dossiers et fichiers dans le répertoire spécifié
    print(f"Contenu de '{dossier}':")
    for element in os.listdir(dossier):
        chemin_element = os.path.join(dossier, element)
        if os.path.isdir(chemin_element):
            print(f"Dossier: {element}")
        else:
            print(f"Fichier: {element}")

# Exemple d'utilisation : spécifier le chemin du dossier à explorer
chemin_dossier = '/kaggle/working/'
lister_dossiers_fichiers(chemin_dossier)


Contenu de '/kaggle/working/':
Dossier: .virtual_documents


In [3]:
import os
import torch
import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)
from transformers import BitsAndBytesConfig
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import nest_asyncio
import locale
locale.getpreferredencoding = lambda: "UTF-8"
from unstructured.partition.pdf import partition_pdf
from llmsherpa.readers import LayoutPDFReader

# Initialize paths
pdf_folder = "/kaggle/input/datasetpdf-fiscabot/PDF Data"
output_folder = "/kaggle/working/"

# Initialize PDF reader
llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_reader = LayoutPDFReader(llmsherpa_api_url)

# Function to process a single PDF
def process_pdf(pdf_path, output_dir):
    try:
        # Extract text from PDF
        doc = pdf_reader.read_pdf(pdf_path)
        full_text = ""

        for chunk in doc.chunks():
            full_text += chunk.to_context_text() + "\n\n"

        # Save extracted text to a .txt file
        base_name = os.path.splitext(os.path.basename(pdf_path))[0]
        output_file_path = os.path.join(output_dir, base_name + '.txt')
        with open(output_file_path, 'w', encoding='utf-8') as f:
            f.write(full_text)

        # Extract images if needed
        partition_pdf(pdf_path)

        print(f"Processed {pdf_path} and saved text to {output_file_path}")
    except KeyError as e:
        print(f"Error processing {pdf_path}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred while processing {pdf_path}: {e}")

# Walk through all subdirectories in the PDF folder
for root, dirs, files in os.walk(pdf_folder):
    for file in files:
        if file.endswith('.pdf'):
            pdf_path = os.path.join(root, file)

            # Create output directory for text files
            base_name = os.path.splitext(file)[0]
            relative_path = os.path.relpath(root, pdf_folder)
            output_dir = os.path.join(output_folder, relative_path, base_name)
            os.makedirs(output_dir, exist_ok=True)

            # Process the PDF
            process_pdf(pdf_path, output_dir)


2024-06-29 13:58:12.006729: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-29 13:58:12.006835: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-29 13:58:12.095077: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Processed /kaggle/input/datasetpdf-fiscabot/PDF Data/TVA (Taxe sur la Valeur Ajoutée)/Taux TVA articles 8 et 7 .pdf and saved text to /kaggle/working/TVA (Taxe sur la Valeur Ajoutée)/Taux TVA articles 8 et 7 /Taux TVA articles 8 et 7 .txt
Processed /kaggle/input/datasetpdf-fiscabot/PDF Data/TVA (Taxe sur la Valeur Ajoutée)/Tableau A .pdf and saved text to /kaggle/working/TVA (Taxe sur la Valeur Ajoutée)/Tableau A /Tableau A .txt
Processed /kaggle/input/datasetpdf-fiscabot/PDF Data/TVA (Taxe sur la Valeur Ajoutée)/Champs d_app TVA art 1 à 4 .pdf and saved text to /kaggle/working/TVA (Taxe sur la Valeur Ajoutée)/Champs d_app TVA art 1 à 4 /Champs d_app TVA art 1 à 4 .txt
Processed /kaggle/input/datasetpdf-fiscabot/PDF Data/TVA (Taxe sur la Valeur Ajoutée)/Restitution TVA art 15.pdf and saved text to /kaggle/working/TVA (Taxe sur la Valeur Ajoutée)/Restitution TVA art 15/Restitution TVA art 15.txt
Processed /kaggle/input/datasetpdf-fiscabot/PDF Data/TVA (Taxe sur la Valeur Ajoutée)/Art 11

In [4]:
# Specify the file path
file_path = "/kaggle/working/Traités Bilatéraux D'investissement/TBI Tunisie-Tchequie/TBI Tunisie-Tchequie.txt"

# Open and read the file
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        print(content)
except FileNotFoundError:
    print(f"The file {file_path} does not exist.")
except Exception as e:
    print(f"An error occurred while reading the file: {e}")


OF INVESTMENTS
The Czech Republic and the Republic of Tunisia (hereinafter referred to as the "Contracting Parties") , Desiring to develop economic cooperation of both States on the basis of equality and mutual benefit, Preoccupied to encourage and create favourable conditions for investments of investors of one State in the territory of the other State, and Conscious that the promotion and reciprocal protection of investments, according to the present Agreement, stimulates the business initiatives in this field, Have agreed as follows : /) ,

OF INVESTMENTS
2 -

OF INVESTMENTS > ARTICLE 1 Defmitions
For the purpose of the present Agreement II The term ,,investment" shall comprise every kind of asset invested by an investor of one Contracting Party in the territory of the other Contracting Party in accordance Mth the laws and regulations of the latter and shall include, in particular, though not exclusively :

OF INVESTMENTS > ARTICLE 1 Defmitions
a) movable and immovable property as w

In [5]:
import shutil

# Chemin du dossier à compresser
folder_path = '/kaggle/working/'
# Chemin du fichier zip de sortie
zip_file = '/kaggle/working/Dataset_Text_format.zip'

# Compresser le dossier
shutil.make_archive(zip_file.replace('.zip', ''), 'zip', folder_path)


'/kaggle/working/Dataset_Text_format.zip'